In [18]:
# Loading all the needed Packages
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB

# Import classes from Class.ipynb
from Class import *

# PROBLEM 1

## 1) Data Handling

In [19]:
## PREPARING THE DATA

# Load all CSV files
Dem_Data = pd.read_excel('../Data/LoadProfile.xlsx', sheet_name='Python_Dem_Data')
Uti_Data = pd.read_excel('../Data/LoadProfile.xlsx', sheet_name='Python_Uti_Data')
Gen_E_Data = pd.read_excel('../Data/Generators_Existing.xlsx', sheet_name='Python_Gen_E_Data')
Gen_N_Data = pd.read_excel('../Data/Generators_New.xlsx', sheet_name='Python_Gen_N_Data')
Gen_E_MaxC_Data = pd.read_excel('../Data/GenerationProfile.xlsx', sheet_name='Python_Gen_E_MaxC_Data')
Gen_N_MaxC_Data = pd.read_excel('../Data/GenerationProfile.xlsx', sheet_name='Python_Gen_N_MaxC_Data')

# Export the needed matrices
Dem = np.array(Dem_Data)    # Demand profile
Uti = np.transpose(np.array(Uti_Data))   # Utility profile
Gen_E_Cost = np.array(Gen_E_Data['Cost'])   # Existing Generators Operational Cost
Gen_N_Cost = np.array(Gen_E_Data['Cost'])  # New Generators Operational Cost
Gen_E_MaxC = np.array(Gen_E_MaxC_Data)  # Maximum Capacity of Existing Generators (Hourly profile if RES, Max capacity otherwise)
Gen_N_MaxC = np.array(Gen_N_MaxC_Data)  # Maximum Capacity of New Generators (Hourly profile if RES, Max capacity otherwise)
Gen_N_MaxInvCap = np.array(Gen_N_Data['MaxInv (MW)'])  # Maximum New Generators Capacity Investment (MW)
Gen_N_InvCost = np.array(Gen_N_Data['C_CapInv ($/MW)'])  # New Generators Investment Cost ($/MW)
Gen_E_Tech = np.array(Gen_E_Data['Technology'])  # Existing Generators Technology
Gen_N_Tech = np.array(Gen_N_Data['Technology'])  # New Generators Technology

# Fix the shape of matrices with only one column    
Gen_E_Cost = Gen_E_Cost.reshape((Gen_E_Cost.shape[0], 1))
Gen_N_Cost = Gen_N_Cost.reshape((Gen_N_Cost.shape[0], 1))
Gen_N_MaxInvCap = Gen_N_MaxInvCap.reshape((Gen_N_MaxInvCap.shape[0], 1))
Gen_N_InvCost = Gen_N_InvCost.reshape((Gen_N_InvCost.shape[0], 1))
Gen_E_Tech = Gen_E_Tech.reshape((Gen_E_Tech.shape[0], 1))
Gen_N_Tech = Gen_N_Tech.reshape((Gen_N_Tech.shape[0], 1))


In [20]:
## DATA INDEX

# Create a Dataframe to store the name of each vector/matrix we will use, their size and their content
Data_df = pd.DataFrame(columns=['Name', 'Size', 'Content'])
Data_df['Name'] = ['Dem', 'Uti', 'Gen_E_Cost', 'Gen_N_Cost', 'Gen_E_MaxC', 'Gen_N_Max_C', 'Gen_N_MaxInvCap', 'Gen_N_InvCost', 'Gen_E_Tech', 'Gen_N_Tech']
Data_df['Size'] = [Dem.shape, Uti.shape, Gen_E_Cost.shape, Gen_N_Cost.shape, Gen_E_MaxC.shape, Gen_N_MaxC.shape, Gen_N_MaxInvCap.shape, Gen_N_InvCost.shape, Gen_E_Tech.shape, Gen_N_Tech.shape]
Data_df['Content'] = ['Demand for each load for each hour of the investment problem',
                     'Utility for each load for one hour',
                     'Unit cost of each existing generator',
                     'Unit cost of each new generator',
                     'Maximum generation profile of each existing energy source for each hour of the investment problem (Hourly profile if RES, Max capacity otherwise)',
                     'Maximum generation profile of each new energy source for each hour of the investment problem (Hourly profile if RES, Max capacity otherwise)',
                     'Maximum capacity investment of each new generator',
                     'Unit Investment cost of each new generator',
                     'Technology of each existing generator',
                     'Technology of each new generator']
Data_df

,Name,Size,Content
0,Dem,"(3600, 17)",Demand for each load for each hour of the inve...
1,Uti,"(17, 1)",Utility for each load for one hour
2,Gen_E_Cost,"(16, 1)",Unit cost of each existing generator
3,Gen_N_Cost,"(16, 1)",Unit cost of each new generator
4,Gen_E_MaxC,"(3600, 16)",Maximum generation profile of each existing en...
5,Gen_N_Max_C,"(3600, 16)",Maximum generation profile of each new energy ...
6,Gen_N_MaxInvCap,"(16, 1)",Maximum capacity investment of each new generator
7,Gen_N_InvCost,"(16, 1)",Unit Investment cost of each new generator
8,Gen_E_Tech,"(16, 1)",Technology of each existing generator
9,Gen_N_Tech,"(16, 1)",Technology of each new generator


In [21]:
## PARAMETERS DEFINITION

# Time
H = 24          # Hours in a day
D = 5           # Typical days in a year
Y = 30          # Years of the investment timeline
N = H*D*Y       # Number of hours in the investment timeline    

# Number of loads and generators
N_dem = len(Dem[0,:])       # Number of loads
N_gen_E = len(Gen_E_Cost)   # Number of existing generators
N_gen_N = len(Gen_N_Cost)   # Number of new generators

# Hyperparameters
B = 1000000000    # Budget for the investment problem

## 2) Optimisation model

### Market Clearing

In [22]:
## CREATE THE PARAMETERS AND DATA OBJECTS
ParametersObj = Parameters(H, D, Y, N, N_dem, N_gen_E, N_gen_N, B)
DataObj = InputData(Dem, Uti, Gen_E_Cost, Gen_N_Cost, Gen_E_MaxC, Gen_N_MaxC, Gen_N_MaxInvCap, Gen_N_InvCost/500, Gen_E_Tech, Gen_N_Tech)

In [23]:
# Ruun the Market Clearing Problem
MarketClearingPB = MarketClearingProblem(ParametersObj, DataObj)

Objective value:  3550323853.5573616
    Hour         Load  Existing generators  Price
0      1  1779.385188          1779.385188   40.0
1      2  1650.595419          1650.595419   40.0
2      3  1571.538513          1571.538513   40.0
3      4  1562.649317          1562.649317   40.0
4      5  1595.064126          1595.064126   40.0
5      6  1620.124075          1620.124075   40.0
6      7  2135.303208          2135.303208   50.0
7      8  2255.132627          2255.132627   75.0
8      9  2196.182934          2196.182934   75.0
9     10  2111.414663          2111.414663   75.0
10    11  2087.396136          2087.396136   75.0
11    12  2193.375357          2193.375357   75.0
12    13  2370.720618          2370.720618   75.0
13    14  2635.375001          2635.375001   75.0
14    15  2859.687964          2859.687964   75.0
15    16  3030.423478          3030.423478   75.0
16    17  3096.497028          3096.497028   75.0
17    18  3151.103269          3151.103269   75.0
18    19  308

### Investment Problem

In [24]:
# Run the investmentmodel
InvestmentPB = InvestmentProblem(ParametersObj, DataObj, MarketClearingPB.res.DA_price)

Objective value:  1000108000.0
   Technology  Invested capacity (MW)
0        Coal                     0.0
1        Coal                     0.0
2        Coal                   200.0
3         Gas                     0.0
4         Gas                     0.0
5        Coal                     0.0
6        Coal                     0.0
7        Wind                     0.0
8        Wind                     0.0
9          PV                     0.0
10       Wind                     0.0
11       Wind                     0.0
12       Wind                     0.0
13        Gas                     0.0
14         PV                     0.0
15       Coal                   200.0
